# The Diet Problem - Dual

The goal of the diet problem is to find the cheapest combination of foods that will satisfy all the daily nutritional requirements of a person.
The diet problem is one of the first optimization problems to be studied back in the 1930's and 40's. It was first motivated by the Army desire to meet the nutritional requirements of the soldier while minimizing the cost. 

One interesting interpretation of the dual problem (see R.Vanderbei for formal definition of dual) is finding equilibrium prices of nutritional pills given prices and nutritional content of natural foods. However in the modern world many people prefer meal replacement drinks to plain single nutrient pills. In our model we maximize revenue from selling this hypotetical cocktail that is made to repalce daily meal.


# Mathematical Model for the Dual Problem

The above optimization problem ( linear programming problem) can be solved with Python linprog solver from Scipy module (scipy.optimize.linprog). This is a description of linprog functionality:

Linear programming: minimize a linear objective function subject to linear equality and inequality constraints.

scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None

this is a link to full description in scipy documentatation:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

We create a python script that loads input data from Excel spreadsheet, transforms it to a form suitable for scipy.optimize.linprog solver, runs scipy.optimize.linprog solver and prints solver output



# Python script

we use pandas read_excel function to read input data

In [48]:
import pandas as pd

we use numpy to perform vector and matrix operations 

In [49]:
import numpy as np

In [50]:
import scipy.optimize as spo

this is a spreadsheet with input data

In [51]:
data_file = 'DietDataFormatted.xlsx'

From this input data we need to prepare matrices and vectors (A,b,c) for linprog input.

First, read nutritional info for each food that we want to be in our daily diet.

In [52]:
df_A = pd.read_excel(data_file,sheet_name='NutritionalInfo',header = 0,index_col = 0)

In [53]:
df_A

,fat,vitA,vitC,protein,calories
broccoli,0.8,70.0,160.2,8.0,74.0
apple,0.5,73.1,7.9,0.3,81.4
oatm.cookie,3.3,2.9,0.1,1.1,81.0
milk,4.7,100.0,2.3,8.1,121.2
chcken,10.8,77.4,2.0,42.2,227.2
omelette,7.3,409.2,0.1,6.7,99.6


convert this DataFrame into numpy array A 

In [54]:
A = df_A.to_numpy() # we transpose  matrix used in primal problem: np.transpose(df_A.to_numpy())

In [55]:
A

array([[8.000e-01, 7.000e+01, 1.602e+02, 8.000e+00, 7.400e+01],
       [5.000e-01, 7.310e+01, 7.900e+00, 3.000e-01, 8.140e+01],
       [3.300e+00, 2.900e+00, 1.000e-01, 1.100e+00, 8.100e+01],
       [4.700e+00, 1.000e+02, 2.300e+00, 8.100e+00, 1.212e+02],
       [1.080e+01, 7.740e+01, 2.000e+00, 4.220e+01, 2.272e+02],
       [7.300e+00, 4.092e+02, 1.000e-01, 6.700e+00, 9.960e+01]])

read daily requirements info for each nutrient

In [56]:
df_b = pd.read_excel(data_file,sheet_name='Requirements',header = 0,index_col = 0)

In [57]:
df_b

,fat,vitA,vitC,protein,calories
lower bounds,20,50,50,50,1000
upper bounds,65,600,300,110,2000


In [58]:
b = -df_b.to_numpy()[1]


In [59]:
b

array([  -65,  -600,  -300,  -110, -2000], dtype=int64)

read food unit costs

In [60]:
df_c = pd.read_excel(data_file,sheet_name='UnitCosts',header = None,index_col = 0)

In [61]:
df_c

,1
0,
broccoli,1.12
apple,1.68
oatm.cookie,0.63
milk,1.61
chcken,5.88
omelette,0.77


create numpy array from unit costs data

In [62]:
c = df_c.to_numpy().flatten()

In [63]:
c

array([1.12, 1.68, 0.63, 1.61, 5.88, 0.77])

In [64]:
bound = (0, 100)
m = b.shape[0]
bounds = ((bound, ) * m)

In [65]:
bounds

((0, 100), (0, 100), (0, 100), (0, 100), (0, 100))

input matrices and vectors A,b,c are ready, we can run linprog solver

In [66]:
res=spo.linprog(b,A_ub=A,b_ub=c,bounds=bounds)

In [67]:
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -16.481504239891855
              x: [ 0.000e+00  0.000e+00  3.422e-03  0.000e+00  7.727e-03]
            nit: 2
          lower:  residual: [ 0.000e+00  0.000e+00  3.422e-03  0.000e+00
                              7.727e-03]
                 marginals: [ 7.298e+01  7.181e+03  0.000e+00  3.016e+01
                              0.000e+00]
          upper:  residual: [ 1.000e+02  1.000e+02  1.000e+02  1.000e+02
                              9.999e+01]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.024e+00  3.731e-03  6.656e-01
                              4.117e+00  0.000e+00]
                 marginals: [-1.861e+00 -0.000e+00 -0.000e+00 -0.000e+00
                             -0.

from the result object we print daily meal overview as follows

In [79]:
print ('**********************************************************')   
print ('cost of meal repalcement drink, optimized:', -res.fun)
print ('**********************************************************')   
print ('these are prices of individual artificial ingredients:')
for j in range(len(df_A.columns)):
    print (df_A.columns[j], ':', res.x[j])

print ('**********************************************************')    
print ('total cost of nutrients in units of foods ($):')    
for j in range(len(df_A.index)):
    food_i = 0.0
    for i in range(len(df_A.columns)):
        food_i += A[j,i]*res.x[i]
    print (df_A.index[j] , ':', food_i)    

**********************************************************
cost of meal repalcement drink, optimized: 16.481504239891855
**********************************************************
these are prices of individual artificial ingredients:
fat : 0.0
vitA : 0.0
vitC : 0.0034217595112273746
protein : 0.0
calories : 0.007727488193261822
**********************************************************
total cost of nutrients in units of foods ($):
broccoli : 1.12
apple : 0.6560494390702086
oatm.cookie : 0.6262687196053303
milk : 0.9444416158991558
chcken : 1.7625288365315406
omelette : 0.7700000000000001
